### 框架干的最厉害的一件事就是帮我们把返向传播全部计算好了

In [1]:
import torch

需要求导的，可以手动定义：

In [2]:
#方法1
x = torch.randn(3,4,requires_grad=True)
x

tensor([[ 0.1171, -0.2744,  0.2964,  0.8785],
        [-1.2234, -3.2207, -2.1018, -0.5976],
        [ 0.8810,  0.1600, -0.8552,  1.3371]], requires_grad=True)

In [3]:
#方法2
x = torch.randn(3,4)
x.requires_grad=True
x

tensor([[-0.5334, -1.1094,  0.1033, -1.7253],
        [-0.1094, -1.2597,  0.4344,  1.0042],
        [-0.0474, -1.1557, -0.6138, -0.7082]], requires_grad=True)

In [4]:
b = torch.randn(3,4,requires_grad=True)

In [5]:
t = x + b

In [6]:
y = t.sum()
y

tensor(-2.0352, grad_fn=<SumBackward0>)

In [7]:
y.backward()

In [8]:
b.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

虽然没有指定t的requires_grad但是需要用到它，也会默认的

In [9]:
x.requires_grad, b.requires_grad, t.requires_grad

(True, True, True)

### 举个例子看一下：

![title](./img/2.png)

In [10]:
#计算流程
x = torch.rand(1)
b = torch.rand(1, requires_grad = True)
w = torch.rand(1, requires_grad = True)
y = w * x 
z = y + b 

In [11]:
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad#注意y也是需要的

(False, True, True, True)

In [12]:
x.is_leaf, w.is_leaf, b.is_leaf, y.is_leaf, z.is_leaf

(True, True, True, False, False)

返向传播计算

In [13]:
z.backward(retain_graph=True)#如果不清空会累加起来

In [14]:
w.grad

tensor([0.2270])

In [15]:
b.grad

tensor([1.])

### 做一个线性回归试试水

构造一组输入数据X和其对应的标签y

In [19]:
import numpy as np
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
x_train.shape

(11, 1)

In [20]:
y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [21]:
import torch
import torch.nn as nn

### 线性回归模型
- 其实线性回归就是一个不加激活函数的全连接层

In [22]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

In [23]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

In [24]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

指定好参数和损失函数

In [25]:
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

训练模型

In [26]:
for epoch in range(epochs):
    epoch += 1
    # 注意转行成tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    # 梯度要清零每一次迭代
    optimizer.zero_grad() 

    # 前向传播
    outputs = model(inputs)

    # 计算损失
    loss = criterion(outputs, labels)

    # 返向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()
    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.1370353251695633
epoch 100, loss 0.07815976440906525
epoch 150, loss 0.04457935318350792
epoch 200, loss 0.025426404550671577
epoch 250, loss 0.01450230460613966
epoch 300, loss 0.008271547965705395
epoch 350, loss 0.004717752803117037
epoch 400, loss 0.002690848195925355
epoch 450, loss 0.0015347562730312347
epoch 500, loss 0.0008753858273848891
epoch 550, loss 0.0004992824397049844
epoch 600, loss 0.0002847745199687779
epoch 650, loss 0.00016241941193584353
epoch 700, loss 9.264301479561254e-05
epoch 750, loss 5.2842664445051923e-05
epoch 800, loss 3.013893547176849e-05
epoch 850, loss 1.718971907394007e-05
epoch 900, loss 9.80356981017394e-06
epoch 950, loss 5.5925920605659485e-06
epoch 1000, loss 3.188519485775032e-06


### 测试模型预测结果

In [27]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[ 0.9966779],
       [ 2.9971564],
       [ 4.997635 ],
       [ 6.998113 ],
       [ 8.998592 ],
       [10.999071 ],
       [12.999548 ],
       [15.000027 ],
       [17.000505 ],
       [19.000984 ],
       [21.001463 ]], dtype=float32)

### 模型的保存与读取

In [28]:
torch.save(model.state_dict(), 'model.pkl')

In [29]:
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

### 使用GPU进行训练
- 只需要把数据和模型传入到cuda里面就可以了

In [30]:
import torch
import torch.nn as nn
import numpy as np


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


criterion = nn.MSELoss()


learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 1000
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)

    optimizer.zero_grad() 

    outputs = model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.017089178785681725
epoch 100, loss 0.009747050702571869
epoch 150, loss 0.0055593447759747505
epoch 200, loss 0.003170843468979001
epoch 250, loss 0.0018085326300933957
epoch 300, loss 0.001031528227031231
epoch 350, loss 0.0005883394042029977
epoch 400, loss 0.0003355712105985731
epoch 450, loss 0.00019139998767059296
epoch 500, loss 0.00010916833707597107
epoch 550, loss 6.226364348549396e-05
epoch 600, loss 3.5514622140908614e-05
epoch 650, loss 2.025573303399142e-05
epoch 700, loss 1.15537686724565e-05
epoch 750, loss 6.589800705114612e-06
epoch 800, loss 3.7583040466415696e-06
epoch 850, loss 2.1437033410620643e-06
epoch 900, loss 1.222920218424406e-06
epoch 950, loss 6.97348411904386e-07
epoch 1000, loss 3.978846905283717e-07
